In [1]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
import torch
from PIL import Image

A matching Triton is not available, some optimizations will not be enabled
Traceback (most recent call last):
  File "/home/yungshun317/envs/yungshun-py3/lib/python3.12/site-packages/xformers/__init__.py", line 55, in _is_triton_available
    from xformers.triton.softmax import softmax as triton_softmax  # noqa
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/yungshun317/envs/yungshun-py3/lib/python3.12/site-packages/xformers/triton/softmax.py", line 11, in <module>
    import triton
ModuleNotFoundError: No module named 'triton'


In [2]:
!pip3 install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 16.2 MB/s eta 0:00:00m eta 0:00:010:00:01


In [2]:
import cv2

# ControlNet
- https://github.com/lllyasviel/ControlNet
- https://huggingface.co/lllyasviel/sd-controlnet-canny
- https://arxiv.org/pdf/2302.05543
## Conditions
- Sketch
- Normal Map
- Depth Map
- Canny Edge
- M-LSD Line (https://arxiv.org/pdf/2106.00186)
- HED Edge (https://arxiv.org/pdf/1504.06375)
- ADE20k Segmentation (https://people.csail.mit.edu/bzhou/publication/scene-parse-camera-ready.pdf)
- Human Pose

In [3]:
controlnet_canny_model = 'lllyasviel/sd-controlnet-canny'
control_net_canny = ControlNetModel.from_pretrained(controlnet_canny_model, torch_dtype=torch.float16)

In [5]:
pipe = StableDiffusionControlNetPipeline.from_pretrained('runwayml/stable-diffusion-v1-5',
                                                         controlnet=control_net_canny,
                                                         torch_dtype=torch.float16)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
!nvidia-smi

Fri May  3 15:58:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090 ...    Off | 00000000:01:00.0  On |                  N/A |
| N/A   46C    P8               6W /  80W |     77MiB / 16376MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
from diffusers import UniPCMultistepScheduler

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

In [8]:
pipe.enable_attention_slicing()
pipe.enable_xformers_memory_efficient_attention()
pipe.enable_model_cpu_offload()